In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()

True

## load variables
- data folder
- cleaned_df
- json template
- 

In [63]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [64]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

df2

,id,first_name,last_name,gender,age_category,phone_number,email,country,organization,username
0,1831,Muktar Mohamednoor,Rage,Male,18 – 35,252616051599,mrage@so-actionagainsthunger.org,Somalia,ACF,mrage
1,2495,Hassan Omar,Khadhib,Male,,0615807566,khadhiib10@gmail.com,Somalia,Norwegian Refugee Council,Khadhib
2,3228,Jamaal,Tagal,Male,18 – 35,252616719022,suldaanyare1265@gmail.com,Somalia,ACF,JamalAli
3,3726,Abdiaziz Adan,Hassan,Male,18 – 35,0615704046,abdulazizadan02@gmail.com,Somalia,GREDO,guudow
4,4247,Abdikarim,Mohamed,Male,18 – 35,252617144970,in.qooley@gmail.com,Somalia,GREDO SOMALIA,qoley
5,4548,Abdirashid,Sheikh Mohamed,,,0615197528,abdirashid.kassim@gredosom.org,Somalia,Gredo,NRMO


## enumerator
get user details from db for json population

In [65]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1831,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2495,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3228,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_nu

## farmingEntity
- read column values from cleaned df and append to json keys

In [66]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 5,
                     'last_name': 'acf',
                   

## plot

### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

In [89]:
import uuid
import copy

def parse_geoshape(geoshape_str):
    #parse a geoshape string into a list of point dictionaries
    points = []
    parts = geoshape_str.split(';')

    for part in parts:
        part = part.strip()
        if not part:
            continue

        lat, lon, alt, acc = map(float, part.split(' '))

        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })

    return points


final_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #ic(i['KEY'])
    #get only child row for this key
    matching_child_rows = halfmoon_df[halfmoon_df['PARENT_KEY'] == i['KEY']]


    #iterate over matching child is to build crop status
    for x in matching_child_rows.to_dict(orient='records'):
        plot_details['has_crops'] = True if x['crops_present'] == 'yes' else False
        plot_details['crops'] = str(x['crops_in_field']).split(' ')

    # ---- BUILD POINTS FOR THIS ROW ----
    geoshape_str = i['plot_details-polygon']
    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    #--- BUILD ADMINISTRATIVE DETAILS ----
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']

    final_list.append({
        "plot": plot_details
    })
    #print(f"Processed record {count}/{len(records)}")


In [90]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final_list, f, indent=4)

str(halfmoon_df['crops_in_field'][20]).split(' ')

['maize', 'sorghum', 'beans', 'tomato']

## project

In [7]:
project = template[0]['project']
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    project['id'] = idx
    project['project_name'] = i['basic_info-organisation_name']
    ic(project)


ic| project: {'id': 1, 'project_name': 'acf'}
ic| project: {'id': 2, 'project_name': 'concern'}
ic| project: {'id': 3, 'project_name': 'concern'}
ic| project: {'id': 4, 'project_name': 'concern'}
ic| project: {'id': 5, 'project_name': 'acf'}
ic| project: {'id': 6, 'project_name': 'concern'}
ic| project: {'id': 7, 'project_name': 'acf'}
ic| project: {'id': 8, 'project_name': 'concern'}
ic| project: {'id': 9, 'project_name': 'concern'}
ic| project: {'id': 10, 'project_name': 'concern'}
ic| project: {'id': 11, 'project_name': 'concern'}
ic| project: {'id': 12, 'project_name': 'concern'}
ic| project: {'id': 13, 'project_name': 'concern'}
ic| project: {'id': 14, 'project_name': 'concern'}
ic| project: {'id': 15, 'project_name': 'concern'}
ic| project: {'id': 16, 'project_name': 'concern'}
ic| project: {'id': 17, 'project_name': 'concern'}
ic| project: {'id': 18, 'project_name': 'concern'}
ic| project: {'id': 19, 'project_name': 'concern'}
ic| project: {'id': 20, 'project_name': 'concern'}
i

## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [8]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 14 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 81 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 26 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 99 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


#### halfmoon csv - repeat csv
goal: get plot level information from the repeat csv
- get key for current row
- based on key, in parent key colun, iterate to get all data associated with that key.
- 